latest_processed_jobs.csv is the output 
cleaned_linkedin_job_listings_v2.csv is the input

In [2]:
import pandas as pd
import re
import spacy
from spacy.matcher import PhraseMatcher

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Read the data from the CSV file
df = pd.read_csv('cleaned_linkedin_job_listings_v2.csv', encoding='utf-8')

# Predefined list of common skills (you can expand this list)
common_skills = [
    'python', 'java', 'javascript', 'c++', 'c#', 'php', 'ruby', 'swift', 'go', 'scala',
    'react', 'angular', 'vue', 'django', 'flask', 'spring', 'node.js', 'html', 'css',
    'sql', 'nosql', 'mongodb', 'postgresql', 'mysql', 'aws', 'azure', 'git', 'docker',
    'kubernetes', 'machine learning', 'data science', 'tensorflow', 'pytorch', 'nlp',
    'deep learning', 'pandas', 'numpy', 'matplotlib', 'seaborn', 'excel', 'power bi',
    'tableau', 'hadoop', 'spark', 'big data', 'cloud computing', 'devops', 'agile',
    'scrum', 'project management', 'communication skills', 'problem-solving'
]

# Create a PhraseMatcher for skill extraction
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
patterns = [nlp(skill) for skill in common_skills]
matcher.add('SKILLS', None, *patterns)

# Function to clean and preprocess the 'Cleaned_Job_Title' column
def preprocess_text(text):
    if pd.isna(text):
        return ''
    # Remove leading/trailing angle brackets and quotes
    text = text.strip('<>').strip('"').strip("'")
    # Normalize spaces and convert to lowercase
    text = ' '.join(text.lower().split())
    # Remove 'with verification' if present
    text = text.replace('with verification', '')
    return text

# Apply the preprocessing function
df['Cleaned_Job_Title'] = df['Cleaned_Job_Title'].apply(preprocess_text)

# Function to extract structured data from the cleaned text
def extract_fields(text):
    # Initialize fields
    job_title = ''
    company = ''
    location = ''
    skills = set()
    
    if not text:
        return pd.Series({
            'Job_Title': job_title,
            'Company': company,
            'Location': location,
            'Skills': ''
        })
    
    # Split the text by '·' to separate job/company from location
    parts = text.split('·')
    job_and_company = parts[0].strip()
    if len(parts) > 1:
        location = parts[1].strip()
    
    # Extract company name assuming it's the last part after job title
    # Try splitting by ' at ' or ' in ' to find the company name
    job_company_split = re.split(r'\s+(?:at|in)\s+', job_and_company)
    if len(job_company_split) > 1:
        job_title = job_company_split[0].strip()
        company = job_company_split[1].strip()
    else:
        # If 'at' or 'in' not found, assume last word(s) are the company
        job_title_parts = job_and_company.strip().split()
        if len(job_title_parts) > 1:
            company = job_title_parts[-1]
            job_title = ' '.join(job_title_parts[:-1])
        else:
            job_title = job_and_company.strip()
    
    # Use spaCy to extract skills from the job title
    doc = nlp(job_title)
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        skills.add(span.text)
    
    # Convert the set of skills to a semicolon-separated string
    skills_str = '; '.join(sorted(skills))
    
    # Remove the skills from the job title to avoid duplication
    for skill in skills:
        job_title = re.sub(r'\b{}\b'.format(re.escape(skill)), '', job_title, flags=re.IGNORECASE)
    
    # Clean up extra spaces
    job_title = ' '.join(job_title.strip().split())
    
    return pd.Series({
        'Job_Title': job_title,
        'Company': company,
        'Location': location,
        'Skills': skills_str
    })

# Apply the extraction function to create new columns
structured_data = df['Cleaned_Job_Title'].apply(extract_fields)
df = pd.concat([df, structured_data], axis=1)

# Handle missing values in the DataFrame
df.fillna('', inplace=True)

# Display the structured DataFrame
print(df[['Job_Title', 'Company', 'Location', 'Skills']])

# Optionally, save the processed data to a new CSV file
df.to_csv('new.csv', index=False)


                                            Job_Title        Company  \
0   database developer ( server, t- and data wareh...    transimpact   
1   frontend developer internship (remote) (react....        acowale   
2   front end developer intern front end developer...       services   
3   [internship] developer intern [internship] dev...          draft   
4   wordpress - web engineer intern wordpress - we...        limited   
..                                                ...            ...   
73  fullstack web developer ( / mern / ) fullstack...  marketsleague   
74  lab45 - web application developer lab45 - web ...          wipro   
75         frontend developer frontend developer data          sense   
76            fullstack developer fullstack developer       techjays   
77  junior software engineer, trilogy (remote) - $...      crossover   

                                      Location         Skills  
0            pune, maharashtra, india (hybrid)            sql  
1      